In [10]:
# 共通で利用するライブラリ
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Image

warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True, precision=4)
pd.options.display.float_format = '{:.4f}'.format
pd.set_option("display.max_columns", None)
plt.style.use('seaborn')
plt.rcParams["font.size"] = 14
plt.rcParams['font.family'] = 'IPAexGothic'

# この章で利用するライブラリ
from tensorflow.keras import datasets, layers, models
import cv2


In [11]:
def f_imshow(img):
    """画像をNotebook上のインラインに表示する"""
    img = cv2.imencode(".png", img)[1]
    display(Image(img))

# 11章　深層学習に挑戦する10本ノック
ここでは、深層学習を学ぶうえで必要なプログラムを実行していく流れを学ぶ。

前提条件

- 小売店の前を通る道路の画像・映像
  - img01.jpg（道路の通行人の画像（道路全体））
  - img02.jpg（自動車の画像）

### Knock101: 深層学習に必要なデータを準備する

- MNITS
  - 手書き数字画像データ

In [3]:
# データの読み込み

mnist = datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [4]:
# 形状の出力

print(X_train.shape)
print(X_test.shape)

(60000, 28, 28)
(10000, 28, 28)


In [5]:
# 0番目のデータの形状の出力

X_train[0].shape

(28, 28)

In [12]:
# 0番目のデータの表示

f_imshow(X_train[0])

In [13]:
# 0番目のデータの正解データ

y_train[0]

5

In [ ]:
# 深層学習を行うためのデータの変換

# 正則化（各データを最大値で割る。0~1の値に変換する）
X_train_sc, X_test_sc = X_train / 255.0, X_test / 255.0

In [ ]:
# 形状を整える

# 28*28pixelの次元に、チャンネル数の次元を加える
# 28 * 28 * チャンネル数（モノクロなので「1」）
X_train_sc = X_train_sc.reshape((60_000, 28, 28, 1))
X_tets_sc = X_test_sc.reshape((10_000, 28, 28, 1))

### Knock102: 深層学習モデルを構築する